# <center> What is a Feature ? <br><br> Building a Base Dataset for the Analysis of "Feature Issues"</center>

### Glossary

<b>Feature Label</b>: a label that contains the word "feature". <br>
<b>Feature Issue</b>: an issue that has at least one <b>feature label</b>. <br>
<b>Feature Issue Dataset</b>: a dataset containing a number of <b>feature issues</b>. Each feature issue is associated with the repository it comes from through the "repository name" column.

### Importing Required Libraries

In [152]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from github import Github, GithubException
import time
import datetime

### Set Up PyGithub

In [153]:
access_token = 'my_access_token'
g = Github(access_token)

In [154]:
def print_rate_limits():
    # Get the rate limits
    rate_limit = g.get_rate_limit()

    # Print rate limit information
    print("Rate limits:")
    print(f"Core limit: {rate_limit.core.limit}")
    print(f"Core remaining: {rate_limit.core.remaining}")
    print(f"Core reset time: {rate_limit.core.reset}")
    print(f"Search limit: {rate_limit.search.limit}")
    print(f"Search remaining: {rate_limit.search.remaining}")
    print(f"Search reset time: {rate_limit.search.reset}")
    
print_rate_limits()

Rate limits:
Core limit: 5000
Core remaining: 5000
Core reset time: 2023-12-13 20:45:37+00:00
Search limit: 30
Search remaining: 30
Search reset time: 2023-12-13 19:46:37+00:00


### Importing Filtered DataSet

In [155]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

#pd.reset_option('max_columns')
pd.reset_option('max_colwidth')
pd.reset_option('max_rows')

df = pd.read_csv('../../dataset/repositories/filtered/filtered_feature_label_dataset.csv')

# Converting strings to actual python lists
df['labels'] = df['labels'].apply(ast.literal_eval)
df['feature_labels'] = df['feature_labels'].apply(ast.literal_eval)
df['topics'] = df['topics'].apply(ast.literal_eval)

print('Full Shape:', df.shape)

Full Shape: (7899, 34)


In [156]:
df.head(2)

,id,name,isFork,commits,branches,defaultBranch,releases,contributors,license,watchers,stargazers,forks,size,createdAt,pushedAt,updatedAt,homepage,mainLanguage,totalIssues,openIssues,totalPullRequests,openPullRequests,blankLines,codeLines,commentLines,metrics,lastCommit,lastCommitSHA,hasWiki,isArchived,languages,labels,topics,feature_labels
0,3258141,alibaba/vlayout,False,366.0,12.0,master,34.0,17.0,MIT License,341.0,10795,1818,1726,2017-02-27 08:11:47,2021-07-14 04:40:34,2021-07-14 04:40:58,http://tangram.pingguohe.net/,Java,503.0,190.0,23.0,5.0,3672,14074,4262,"[{'blankLines': 3101, 'language': 'Java', 'com...",2021-07-14T04:40:34,00d6be0a1fa89c2f54ca923d2ea2e34d279d289e,NaN,NaN,{'Java': 669506},"[bug, duplicate, enhancement, feature, help wa...",[],[feature]
1,3258277,goxr3plus/java-stream-player,False,181.0,4.0,master,11.0,6.0,GNU General Public License v3.0,9.0,123,29,5664,2017-10-23 02:33:38,2023-02-07 12:28:27,2023-01-24 04:00:42,NaN,Java,43.0,23.0,34.0,7.0,922,2533,1524,"[{'blankLines': 1, 'language': 'YAML', 'commen...",2023-02-07T12:28:27,09ddefe497a894dc9462d330792a9f09f69e2c77,NaN,NaN,{'Java': 130948},"[bug, dependencies, duplicate, enhancement, fe...",[],[feature request]


### Defining Necessary Functions

In [157]:
def build_series_for_issue(repository_name, issue):
    series_data = {
        'repositoryName': repository_name,
        'issueTitle': issue.title,
        'issueBody': issue.body,
        'issueNumber': issue.number,
        'issueState': issue.state,
        'issueCreatedAt': issue.created_at,
        'issueUpdatedAt': issue.updated_at,
        'issueClosedAt': issue.closed_at,
        'issueUser': issue.user.login,
        'assignee': issue.assignee.login if issue.assignee else 'Not assigned',
        # 'closedBy': issue.closed_by.login if issue.closed_by else 'Not closed',
        'issueLabels': [label.name for label in issue.labels],
        'milestone': issue.milestone.title if issue.milestone else 'No milestone',
        'comments': issue.comments,
        'commentsURL': issue.comments_url
    }

    return pd.Series(series_data) if series_data else None

def build_dataframe_for_repository(g, repository_name, associated_feature_labels):
    all_series = []
    
    for feature_label in associated_feature_labels:
        search_query = f'repo:{repository_name} label:{feature_label} type:issue'
        
        try:
            issues = g.search_issues(search_query)
            issues_as_list = list(issues)
            
            for issue in issues_as_list:
                issue_as_series = build_series_for_issue(repository_name, issue)
                if issue_as_series is not None:
                    all_series.append(issue_as_series)
        except Exception as e:
            print(f"Error fetching issues for {repository_name} and label {feature_label}: {e}")

    return pd.concat(all_series, axis=1).T if all_series else None

def build_feature_issue_dataframe(g, df):
    nb_dfs_left_to_build = df.shape[0]
    all_dfs = []

    for index, row in df.iterrows():
        repository_name = row['name']
        feature_labels = row['feature_labels']

        start_time = time.time()
        result_df = build_dataframe_for_repository(g, repository_name, feature_labels)
        end_time = time.time()
        elapsed_time = end_time - start_time
        nb_dfs_left_to_build = nb_dfs_left_to_build - 1

        rate_limit = g.get_rate_limit()
        print(f"Repository '{repository_name}' took {elapsed_time:.4f}s to convert. {nb_dfs_left_to_build} Repositories Left to Build !")
        print(f"You Have {rate_limit.search.remaining} Searches Left")

        if result_df is not None:
            all_dfs.append(result_df)

    return pd.concat(all_dfs, ignore_index=True).dropna(axis=1, how='all') if all_dfs else pd.DataFrame()

def build_issue_dataset_from_repository_dataset_for_range(start, end, df):
    df_sample = df.iloc[start:end]
    start_time = time.time()
    feature_issue_df = build_feature_issue_dataframe(g, df_sample)
    end_time = time.time()
    elapsed_time = end_time - start_time

    delta = datetime.timedelta(seconds=int(elapsed_time))
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    elapsed_time_formatted = f"{hours:02}h{minutes:02}m{seconds:02}s"

    print(f"In total, it Took {elapsed_time_formatted} to Build the Entire 'Feature Issue' Dataset from {df_sample.shape[0]} Repositories")
    return feature_issue_df

### Building "Feature Issue Datasets" (This Part Takes a Very Long Time to Execute ! Do not Repeat this Step More Than Once !)

In [176]:
start = 7000
end = 8000

feature_issue_df = build_issue_dataset_from_repository_dataset_for_range(start, end, df)
feature_issue_df.to_csv(f'../../dataset/issues/partial_datasets/feature_issue_dataset_{start}_{end}.csv', index=False)

Repository 'zdoom/raze' took 0.3976s to convert. 898 Repositories Left to Build !
You Have 29 Searches Left
Repository 'zaykenyon/valorant-rank-yoinker' took 0.5327s to convert. 897 Repositories Left to Build !
You Have 28 Searches Left
Repository 'walletconnect/web3modal' took 3.8225s to convert. 896 Repositories Left to Build !
You Have 23 Searches Left
Repository 'eslint-functional/eslint-plugin-functional' took 0.9138s to convert. 895 Repositories Left to Build !
You Have 21 Searches Left
Repository 'alchemyplatform/alchemy-sdk-js' took 0.4452s to convert. 894 Repositories Left to Build !
You Have 20 Searches Left
Repository 'robinblomberg/kysely-codegen' took 0.4473s to convert. 893 Repositories Left to Build !
You Have 19 Searches Left
Repository 'ssbc/go-ssb' took 0.5641s to convert. 892 Repositories Left to Build !
You Have 18 Searches Left
Repository 'databricks/terraform-provider-databricks' took 7.5611s to convert. 891 Repositories Left to Build !
You Have 9 Searches Left
Er

Request GET /search/issues?q=repo%3Abanchanart%2Fbanchan+label%3Afeature+type%3Aissue failed with 403: Forbidden
Setting next backoff to 35.454975s


Repository 'banchanart/banchan' took 39.4717s to convert. 884 Repositories Left to Build !
You Have 25 Searches Left
Repository 'fluttercandies/flutter_image_compress' took 0.4432s to convert. 883 Repositories Left to Build !
You Have 24 Searches Left
Error fetching issues for fluencelabs/rust-peer and label feature: 422 {"message": "Validation Failed", "errors": [{"message": "The listed users and repositories cannot be searched either because the resources do not exist or you do not have permission to view them.", "resource": "Search", "field": "q", "code": "invalid"}], "documentation_url": "https://docs.github.com/v3/search/"}
Repository 'fluencelabs/rust-peer' took 0.4490s to convert. 882 Repositories Left to Build !
You Have 23 Searches Left
Repository 'buckyos/cyfs' took 2.3972s to convert. 881 Repositories Left to Build !
You Have 20 Searches Left
Repository 'vectordotdev/vector' took 0.4452s to convert. 880 Repositories Left to Build !
You Have 19 Searches Left


Request GET /search/issues?q=repo%3Adatafuselabs%2Fdatabend+label%3Ac-feature+type%3Aissue&page=20 failed with 403: Forbidden
Setting next backoff to 36.635303s


Repository 'datafuselabs/databend' took 64.9245s to convert. 879 Repositories Left to Build !
You Have 14 Searches Left


...

Repository 'apecloud/kubeblocks' took 60.6269s to convert. 26 Repositories Left to Build !
You Have 13 Searches Left
Repository 'microsoft/semantic-kernel' took 0.4425s to convert. 25 Repositories Left to Build !
You Have 12 Searches Left
Repository 'meshery/meshery' took 1.4003s to convert. 24 Repositories Left to Build !
You Have 10 Searches Left
Repository 'bitnami/containers' took 7.9834s to convert. 23 Repositories Left to Build !
You Have 0 Searches Left


Request GET /search/issues?q=repo%3AMountainDash%2Fnonebot-bison+label%3Afeature+type%3Aissue failed with 403: Forbidden
Setting next backoff to 36.169281s


Repository 'MountainDash/nonebot-bison' took 37.0321s to convert. 22 Repositories Left to Build !
You Have 29 Searches Left
Repository 'dw-0/kiauh' took 0.4559s to convert. 21 Repositories Left to Build !
You Have 28 Searches Left
Repository 'detekt/sonar-detekt' took 0.5982s to convert. 20 Repositories Left to Build !
You Have 27 Searches Left
Repository 'krakend/krakend-ce' took 0.4489s to convert. 19 Repositories Left to Build !
You Have 26 Searches Left


Request GET /search/issues?q=repo%3Ahackforla%2Fwebsite+label%3Afeature%3A+tables+type%3Aissue failed with 403: Forbidden
Setting next backoff to 45.482s
Request GET /search/issues?q=repo%3Ahackforla%2Fwebsite+label%3Ap-feature%3A+sdg+page+type%3Aissue failed with 403: Forbidden
Setting next backoff to 47.021915s


Repository 'hackforla/website' took 120.6610s to convert. 18 Repositories Left to Build !
You Have 25 Searches Left
Repository 'tsparticles/tsparticles' took 2.1263s to convert. 17 Repositories Left to Build !
You Have 23 Searches Left
Repository 'open-sauced/app' took 0.4555s to convert. 16 Repositories Left to Build !
You Have 22 Searches Left
Repository 'MixedRealityToolkit/MixedRealityToolkit-Unity' took 0.4466s to convert. 15 Repositories Left to Build !
You Have 21 Searches Left
Repository 'tinkerbell/smee' took 0.7171s to convert. 14 Repositories Left to Build !
You Have 20 Searches Left
Repository 'icosa-foundation/open-brush' took 0.4438s to convert. 13 Repositories Left to Build !
You Have 19 Searches Left
Repository 'erigontech/silkworm' took 0.4635s to convert. 12 Repositories Left to Build !
You Have 18 Searches Left
Repository 'paradigmxyz/rivet' took 0.4547s to convert. 11 Repositories Left to Build !
You Have 17 Searches Left
Repository 'jacoscaz/quadstore' took 0.6908s